# Textklassifikation, Teil 2

Im ersten Teil haben wir Texte aus dem Datensatz *GermEval 2018* mithilfe von Frequency-Vektoren und einem Naive-Bayes-Klassifikator klassifiziert.

Dieses Mal verwenden wir ein vortrainiertes *Word2Vec* Modell und wollen untersuchen, ob wir damit bessere Ergebnisse erzielen.

## Daten einlesen

Wie beim letzten Mal lesen wir zunächst die Daten ein.

In [7]:
import pandas as pd

train_ds = pd.read_csv('../data/GermEval-2018/germeval2018.training.txt', sep='\t', header=None, names=['text', 'coarse', 'fine'])
test_ds = pd.read_csv('../data/GermEval-2018/germeval2018.test.txt', sep='\t', header=None, names=['text', 'coarse', 'fine'])

In [6]:
train_ds

,text,coarse,fine
0,"@corinnamilborn Liebe Corinna, wir würden dich...",OTHER,OTHER
1,@Martin28a Sie haben ja auch Recht. Unser Twee...,OTHER,OTHER
2,@ahrens_theo fröhlicher gruß aus der schönsten...,OTHER,OTHER
3,@dushanwegner Amis hätten alles und jeden gewä...,OTHER,OTHER
4,@spdde kein verläßlicher Verhandlungspartner. ...,OFFENSE,INSULT
...,...,...,...
5004,Gegens. Zul. zu Patenamt &amp; gegenseitige An...,OTHER,OTHER
5005,"@GlasenappHenrik Zu Merkel fällt mir nur ein, ...",OFFENSE,INSULT
5006,@KokoLores20 @krippmarie Ein richtiges Zeichen...,OFFENSE,ABUSE
5007,"@Hartes_Geld ,Honecker‘Merkel macht uns zur ,D...",OFFENSE,ABUSE


## Texte bereinigen

Auch die Bereinigung der Texte hatten wir schon beim letzten Mal.

In [53]:
import re

def clean_tweet(text):
    """ Preprocess and tokenize a tweet. """
    
    # remove handles, i.e. @username
    text = re.sub('\@\w+', '', text)

    # remove hashtags, quotes, etc.
    text = re.sub('[\#"\']+', '', text)
    
    # replace hyphens with blanks
    text = text.replace('-', ' ')
    return text

In [16]:
train_ds['text'] = train_ds['text'].map(clean_tweet)
test_ds['text'] = test_ds['text'].map(clean_tweet)

In [59]:
!pip install gensim nltk

## Word2Vec-Modell laden

Die Datei `twitter-de_d100_w5_min10.bin` enthält ein mit deuschen Twitternachrichten trainiertes *Word2Vec* Modell, das wir nun laden.

In [60]:
import gensim 

w2v = gensim.models.KeyedVectors.load_word2vec_format('/data/nlp/twitter-de_d100_w5_min10.bin', binary=True)

## Aufgabe 1: Wortähnlichkeiten

Der Vorteil von Word2Vec ist, dass die generierten Embeddings die Bedeutung von Worten widerspiegeln.
Nutzen Sie die Methode `most_similar` des Modells, um ähnliche Worte für folgende Begriffe zu finden:

- Stümper
- Merkel
- Hetze

In [49]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics


def evaluate(classifier):
    """Evaluiere einen classifier auf den Testdaten"""
    predicted = classifier.predict(test_ds['text'])
    print(f"Confusion matrix:\n{metrics.confusion_matrix(test_ds['coarse'], predicted)}")
    print(f"{metrics.classification_report(test_ds['coarse'], predicted)}")
    return np.mean(predicted == test_ds['coarse'])
    

## Aufgabe 2: Wortvektoren zur Textklassifikation

Wir wollen nun untersuchen, inwiefern die Wortvektoren uns bei der Klassifikation der Tweets helfen. Die folgende Klasse 
`EmbeddingVectorizer` ordnet jedem Text den "Durchnittsvektor" seiner Worte zu. Dazu verwendet sie die Methode `get_mean_vector`.

Verwenden Sie den `EmbeddingVectorizer` und einen `LinearSVC`, um die Tweets zu klassifizieren.

Trainieren Sie den Klassifikator auf den Trainingsdaten und evaluieren Sie ihn auf den Testdaten. 
Wie gut ist Ihr Ergebnis? Vergleichen Sie Ihr Ergebnis mit dem Ergebnis vom letzten Mal und den [Ergebnissen des GermEval-2018](../data/GermEval-2018/results.pdf).

In [43]:
from sklearn.base import BaseEstimator
import nltk
import numpy as np

class EmbeddingVectorizer(BaseEstimator):
    """Convert a collection of text documents to a matrix of vectors created from word embeddings """
    
    def __init__(self, model):
        self.model = model
        self.tokenizer = nltk.tokenize.casual.TweetTokenizer()
    
        
    def fit(self, X, y, **fit_params):
        """Nothing to do here, we use a pre-trained model. """
        return self
    
    def transform(self, raw_documents):
        """Transform documents to embedding matrix by mean of the embeddings
        of individual words.
        """
        if isinstance(raw_documents, str):
            raise ValueError(
                "Iterable over raw text documents expected, "
                "string object received.")

        _X = []
        for doc in raw_documents:
            words = self.tokenizer.tokenize(doc)
            _X.append(self.model.get_mean_vector(words))
            
        X = np.array(_X)
        return X

In [50]:
vectorizer = EmbeddingVectorizer(w2v)